**10.** Non-Bayesian inference: replicate the analysis of the bioassay example in Section 3.7
using non-Bayesian inference. This problem does not have a unique answer, so be clear
on what methods you are using.

**(a)** Construct an ‘estimator’ of $(α, β)$; that is, a function whose input is a dataset, $(x, n, y)$,
and whose output is a point estimate $(\hat{\alpha}, \hat{\beta})$. Compute the value of the estimate for
the data given in Table 5.2.

**(b)** The bias and variance of this estimate are functions of the true values of the parameters
$(α, β)$ and also of the sampling distribution of the data, given $α$, $β$. Assuming the
binomial model, estimate the bias and variance of your estimator.

**(c)** Create approximate 95% confidence intervals for $α$, $β$, and the LD50 based on asymptotic
theory and the estimated bias and variance.

**(d)** Does the inaccuracy of the normal approximation for the posterior distribution (compare
Figures 3.3 and 4.1) cast doubt on the coverage properties of your confidence
intervals in (c)? If so, why?

**(e)** Create approximate 95% confidence intervals for $α$, $β$, and the LD50 using the jackknife
or bootstrap (see Efron and Tibshirani, 1993).

**(f)** Compare your 95% intervals for the LD50 in (c) and (e) to the posterior distribution
displayed in Figure 3.4 and the posterior distribution based on the normal approximation,
displayed in 4.2b. Comment on the similarities and differences among the four
intervals. Which do you prefer as an inferential summary about the LD50? Why?

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from more_itertools import flatten
import scipy.optimize
from firthlogist import FirthLogisticRegression

In [2]:
data = pd.DataFrame({'x': [-0.86, -0.30, -0.05, 0.73], 'n': [5, 5, 5, 5], 'y': [0, 1, 3, 5]})
data = data.assign(w = data['y'] / data['n'])
data_ext = pd.DataFrame({'y': [0,0,0,0,0,1,0,0,0,0,1,1,1,0,0,1,1,1,1,1],
                         'x': np.repeat([-0.86, -0.30, -0.05, 0.73], 5)})

#### Fit with ordinary logistic regression

In [3]:
fit = smf.glm("y ~ x", data = data_ext, family=sm.families.Binomial()).fit()
print(fit.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                   20
Model:                            GLM   Df Residuals:                       18
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -5.8944
Date:                Tue, 30 Aug 2022   Deviance:                       11.789
Time:                        15:00:07   Pearson chi2:                     10.4
No. Iterations:                     7   Pseudo R-squ. (CS):             0.5447
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8466      1.019      0.831      0.4

In [4]:
theta_hat = [*list(fit.params), -fit.params[0] / fit.params[1]]

#### Normal approximation

In [5]:
ci = fit.conf_int(alpha = 0.5).rename(columns = {0: '2.5%', 1: '97.5%'})
print(ci)

               2.5%      97.5%
Intercept  0.159223   1.533938
x          4.462240  11.035394


#### Bootstrapping

In [11]:
B = 1000
n = data_ext.shape[0]
theta_star = np.zeros((B, 3))
for b in range(B):
    ind = np.random.choice(np.arange(n), size = n, replace = True)
    data_i = data_ext.loc[ind]
    fit = FirthLogisticRegression(max_iter = 150)
    fit.fit(data_i['x'].to_numpy().reshape(-1, 1), data_i['y'])
    theta_star[b, :] = [fit.intercept_, *fit.coef_, *-fit.intercept_ / fit.coef_]

In [12]:
bias = (theta_star - theta_hat).mean(axis = 0)
Std = np.std(theta_star, axis = 0)

In [13]:
print("\n{:^10} | {:^10} | {:^16}".format("", "2.5%", "97.5%"))
print("*"*40)
for i, label in zip(range(3), ('intercept', 'x', 'LD50')):
    boot_ci = np.quantile(theta_star[:, i], q = [0.025, 0.975], axis = 0)
    print("{:<10} | {:^10} | {:^16}".format(label, round(boot_ci[0], 3), round(boot_ci[1], 3)))


           |    2.5%    |      97.5%      
****************************************
intercept  |   -0.962   |      2.945      
x          |   3.065    |      15.405     
LD50       |   -0.325   |      0.215      


#### Normal approximation for the posterior

In [14]:
data_ext.insert(1, 'dummy', 1)

In [15]:
def logistic_extended(beta, y, x):
    return -((-np.log(1 + np.exp(np.dot(x, beta)))).sum() + (y*(np.dot(x, beta))).sum())

def logistic_aggregated(w, df):
    z = w[0] + w[1] * df['x'].values
    return -np.sum(df['y'].values * z - df['n'].values * np.log1p(np.exp(z)))

In [16]:
theta_start = [0, 0]
mle_ext = scipy.optimize.minimize(logistic_extended, theta_start, args=(data_ext['y'], data_ext.loc[:, 'dummy':]), method='BFGS')

In [17]:
mle = scipy.optimize.minimize(logistic_aggregated, theta_start, args=(data))
print(mle)

      fun: 5.894441638970639
 hess_inv: array([[ 0.96949024,  3.4468913 ],
       [ 3.4468913 , 23.51510283]])
      jac: array([ 5.48362732e-06, -2.38418579e-07])
  message: 'Optimization terminated successfully.'
     nfev: 42
      nit: 13
     njev: 14
   status: 0
  success: True
        x: array([0.84658517, 7.74883075])


In [18]:
SE = np.sqrt(np.diag(mle.hess_inv))
ci_norm_post = [mle.x - 1.96 * SE, mle.x + 1.96 * SE]

In [19]:
print("\n{:^10} | {:^10} | {:^16}".format("", "2.5%", "97.5%"))
print("*"*40)
for i, label in zip(range(2), ('alpha', 'beta')):
    print("{:<10} | {:^10} | {:^16}".format(label, round(ci_norm_post[0][i], 3), round(ci_norm_post[1][i], 3)))


           |    2.5%    |      97.5%      
****************************************
alpha      |   -1.083   |      2.776      
beta       |   -1.756   |      17.253     


#### Posterior directly by sampling

In [20]:
A = np.linspace(-4, 8, 50)
B = np.linspace(-10, 40, 50)
cA = np.repeat(A, len(B))
cB = np.repeat(B, len(A))

In [21]:
def logl(df, a, b):
    return df['y'].values * (a + b * df['x'].values) - df['n'] * np.log1p(np.exp(a + b * df['x'].values))

In [22]:
nsamp = 1000
p = [np.exp(np.sum(logl(data, a, b))) for a in A for b in B]
samp_indices = np.random.choice(np.arange(len(p)), size = nsamp, replace = True, p = p / np.sum(p))

In [23]:
samp_A = cA[samp_indices[:nsamp]]
samp_B = cB[samp_indices[:nsamp]]
samp_A = samp_A + np.random.uniform((A[0] - A[1])/2, (A[1] - A[0])/2, size = nsamp)
samp_B = samp_B + np.random.uniform((B[0] - B[1])/2, (B[1] - B[0])/2, size = nsamp)
ld50 = -samp_A / samp_B

In [24]:
samps = {'alpha': samp_A, 'beta': samp_B, 'LD50': ld50}

In [25]:
print("\n{:^10} | {:^10} | {:^16}".format("", "2.5%", "97.5%"))
print("*"*40)
for k, v in samps.items():
    print("{:<10} | {:^10} | {:^16}".format(k, *np.round(np.quantile(v, q = [0.025, 0.975]), 2)))


           |    2.5%    |      97.5%      
****************************************
alpha      |   -0.56    |       3.67      
beta       |    4.48    |      21.83      
LD50       |   -0.17    |       0.13      
